# Abstract Base Classes & Operator Overloading

### Loading Libraries

In [83]:
# Math
import math
from math import hypot

# OS
import re
import abc
import time
import random
import bisect

from decimal import Decimal

# Types & Annotations
from __future__ import annotations
from collections.abc import Container, Mapping, Hashable
from typing import Hashable, Mapping, TypeVar, Any, overload, Union, Sequence, Dict
from typing import List, Protocol, NoReturn, Union, Set, Tuple, Optional, Iterable, Iterator, cast

# Dunctional Tools
from functools import wraps

# Files & Path
import logging
import zipfile
import fnmatch
from pathlib import Path
from urllib.request import urlopen

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn
import matplotlib.pyplot as plt

In [52]:
Comparable = TypeVar('Comparable')

BaseMapping = Mapping[Comparable, Any]

### Creating an Abstract Base Class

In [2]:
class MediaLoader(abc.ABC):
    @abc.abstractmethod
    def play(self) -> None:
        ...

    @property
    @abc.abstractmethod
    def ext(self) -> str:
        ...

In [3]:
MediaLoader.__abstractmethods__

frozenset({'ext', 'play'})

In [4]:
class Wav(MediaLoader):
    pass

In [5]:
x = Wav()

TypeError: Can't instantiate abstract class Wav with abstract methods ext, play

In [6]:
class Ogg(MediaLoader):
    ext = '.ogg'
    def play(self):
        pass

In [7]:
o = Ogg()

### The ABC of Collections

In [8]:
Container.__abstractmethods__

frozenset({'__contains__'})

In [9]:
help(Container.__contains__)

Help on function __contains__ in module collections.abc:

__contains__(self, x)



In [10]:
class OddIntegers:
    def __contains__(self, x: int) -> bool:
        return x % 2 != 0

In [11]:
odd = OddIntegers()

In [12]:
isinstance(odd, Container)

True

In [13]:
issubclass(OddIntegers, Container)

True

In [14]:
odd = OddIntegers()

In [15]:
1 in odd

True

In [16]:
2 in odd

False

In [17]:
3 in odd

True

### The `collections.abc` Module

In [18]:
Comparable = TypeVar('Comparable')

BaseMapping = Mapping[Comparable, Any]

In [19]:
x = dict({"a": 42, "b": 7, "c": 6})

y = dict([("a", 42), ("b", 7), ("c", 6)])

In [20]:
x == y

True

In [21]:
BaseMapping = Mapping[Comparable, Any]
class Lookup(BaseMapping):
    @overload
    def __init__(self, source: Iterable[tuple[Comparable, Any]]) -> None:
        ...

    @overload
    def __init__(self, source: BaseMapping) -> None:
        ...

    def __init__(self, source: Union[Iterable[tuple[Comparable, Any]], BaseMapping, None] = None,) ->None:
        sorted_pairs: Sequence[tuple[Comparable, Any]]
        if isinstance(source, Sequence):
            sorted_pairs = sorted(source)
        elif isinstance(source, abc.Mapping):
            sorted_pairs = sorted(source.items())
        else:
            sorted_pairs = []
        self.key_list = [p[0] for p in sorted_pairs]
        self.value_list = [p[1] for p in sorted_pairs]

    def __len__(self) -> int:
        return len(self.key_list)

    def __iter__(self) -> Iterator[Comparable]:
        return iter(self.key_list)

    def __contains__(self, key: object) -> bool:
        index = bisect.bisect_left(self.key_list, key)
        return key == self.key_list[index]

    def __getitem__(self, key: Comparable) -> Any:
        index = bisect.bisect_left(self.key_list, key)
        if key == self.key_list[index]:
            return self.value_list[index]
        raise KeyError(key)

In [22]:
x = Lookup(
    [
        ["z", "Zillah"],
        ["a", "Amy"],
        ["c", "Clara"],
        ["b", "Basil"],
    ]
)

In [23]:
x["c"]

'Clara'

In [24]:
x["m"] = "Maud"

TypeError: 'Lookup' object does not support item assignment

In [25]:
class Comparable(Protocol):
    def __eq__(self, other:Any) -> bool: ...
    def __ne__(self, other:Any) -> bool: ...
    def __le__(self, other:Any) -> bool: ...
    def __lt__(self, other:Any) -> bool: ...
    def __ge__(self, other:Any) -> bool: ...
    def __gt__(self, other:Any) -> bool: ...

In [26]:
class Die (abc.ABC):
    def __init__(self) -> None:
        self.face: int
        self.roll()

    @abc.abstractmethod
    def roll(self) -> None:
        ...

    def __repr__(self) -> str:
        return f"{self.face}"

In [27]:
"""
Messy Code Sample as follow:
"""

# class Bad(Die):
#     def roll(self, a: int, b: int) -> float:
#         return (a+b)/2

'\nMessy Code Sample as follow:\n'

In [28]:
class D4(Die):
    def roll(self) -> None:
        self.face = random.choice((1, 2, 3, 4))

In [29]:
class D6(Die):
    def roll(self) -> None:
        self.face = random.randint(1, 6)

In [30]:
class Dice(abc.ABC):
    def __init__(self, n: int, die_class: Type[Die]) -> None:
        self.dice = [die_class() for _ in range(n)]

    @abc.abstractmethod
    def roll(self) -> None:
        ...

    @property
    def total(self) -> int:
        return sum(d.face for d in self.dice)

In [31]:
class SimpleDice(Dice):
    def roll(self) -> None:
        for d in self.dice:
            d.roll()

In [32]:
sd = SimpleDice(6, D6)

In [33]:
sd.roll()

In [34]:
sd.total

22

In [35]:
class YatchDice(Dice):
    def __init__(self) -> None:
        super().__init__(5, D6)
        self.saved: Set[int] = set()

    def saving(self, positions: Iterable[int]) -> "YatchDice":
        if not all(0 <= n < 6 for n in positions):
            raise ValueError("Invalid position")
        self.saved = set(positions)
        return self

    def roll(self) -> None:
        for n, d in enumerate(self.dice):
            if n not in self.saved:
                d.roll()
        self.saved = set()

In [36]:
sd = YatchDice()

In [37]:
sd.roll()

In [38]:
sd.dice

[3, 3, 4, 5, 2]

In [39]:
sd.saving([0, 1, 2]).roll()

In [40]:
sd.dice

[3, 3, 4, 4, 2]

### Demystifying The Magic

In [41]:
# from dice import Die

In [42]:
Die.__abstractmethods__

frozenset({'roll'})

In [43]:
Die.roll.__isabstractmethod__

True

In [53]:
class DieM(metaclass=abc.ABCMeta):
    def __init__(self) -> None:
        self.face: int
        self.roll()

    @abc.abstractmethod
    def roll(self) -> None:
        ...

### Operator Overloading

In [54]:
def len(object: Sized) -> int:
    return object.__len__()

In [55]:
home = Path.home()

In [56]:
home / "miniconda3" / "envs"

PosixPath('/Users/isisromero/miniconda3/envs')

In [58]:
class DDice:
    def __init__(self, *die_class: Type[Die]) -> None:
        self.dice = [dc() for dc in die_class]
        self.adjust: int = 0

    def plus(self, adjust: int = 0) -> "DDice":
        self.adjust = adjust
        return self

    def roll(self) -> None:
        for d in self.dice:
            d.roll()

    @property
    def total(self) -> int:
        return sum(d.face for d in self.dice) + self.adjust

In [60]:
def __add__(self, die_class: Any) -> "DDice":
    if isinstance(die_class, type) and insubclass(die_class, Die):
        new_classes = [type(d) for d in self.dice] + [die_class]
        new = DDice(*new_classes).plus(self.adjust)
        return new
    elif isinstance(die_class, int):
        new_classes = [type(d) for d in self.dice]
        new = DDice(*new_classes).plus(die_class)
        return new
    else:
        return NotImplemented

def __radd__(self, die_class: Any) -> "DDice":
    if isinstance(die_class, type) and issubclass(die_class, Die):
        new_classes = [die_class] + [type(d) for d in self.dice]
        new = DDice(*new_classes).plus(self.adjust)
        return new
    elif isinstance(die_class, int):
        new_classes = [type(d) for d in self.dice]
        new = DDice(*new_classes).plus(die_class)
        return new
    else:
        return NotImplemented
    

In [61]:
def __mul__(self, n: Any) -> "DDice":
    if isinstance(n, int):
        new_classes = [type(d) for d in self.dice for _ in range(n)]
        return DDice(*new_classes).plus(self.adjust)
    else:
        return NotImplemented

def __rmul__(self, n: Any) -> "DDice":
    if instance(n, int):
        new_classes = [type(d) for d in self.dice for _ in range(n)]
        return DDice(*new_classes).plus(self.adjust)
    else:
        return NotImplemented

In [64]:
y = DDice(D6, D6)

In [66]:
# y += D6

In [68]:
def __iadd__(self, die_class: Any) -> "DDice":
    if isinstance(die_class, type) and issubclass(die_class, Die):
        self.dice += [die_class()]
        return self
    elif isinstance(die_class, int):
        self.adjust += die_class
        return self
    else:
        return NotImplemented

In [69]:
y = DDice(D6, D6)

In [73]:
# y += D6

In [75]:
# y += 2

### Extending Built-ins

In [78]:
d = {"a": 42, "a": 3.14}

In [79]:
d

{'a': 3.14}

In [80]:
{1: "one", True: "true"}

{1: 'true'}

In [85]:
class NoDupDict(Dict[Hashable, Any]):
    def __setitem__(self, key, value) -> None:
        if key in self:
            raise ValueError(f"duplicate {key!r}")
        super().__setitem__(key, value)

In [86]:
nd = NoDupDict()

In [89]:
nd["a"] = 1
nd["a"] = 2

ValueError: duplicate 'a'

In [90]:
NoDupDict({"a": 42, "a": 3.14})

{'a': 3.14}

In [91]:
DictInit = Union[
    Iterable[Tuple[Hashable, Any]],
    Mapping[Hashable, Any],
    None]

In [92]:
class NonDupDict(Dict[Hashable, Any]):
    def __setitem__(self, key: Hashable, value: Any) -> None:
        if key in self:
            raise ValueError(f"duplicate {key!r}")
        super().__setitem__(key, value)

    def __init__(self, init: DictInit = None, **kwargs: Any) -> None:
        if isinstance(init, Mapping):
            super().__init__(init, **kwargs)
        elif isisntance(init, **kwargs):
            for k, v in cast(Iterable[Tuple[Hashable, Any]], init):
                self[k] = v
        elif init is None:
            super().__init__(**kwargs)
        else:
            super().__init__(init, **kwargs)

    

### Metaclasses